<a href="https://colab.research.google.com/github/CleitonValandro/predictive-model-in-tumor-samples-using-artificial-intelligence/blob/task%2Fmachine-learning/PredictiveModelInTumorSamplesUsingArtificialIntelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação dos dados do TCGA via Xena Browser**

In [ ]:
#xenaPython é uma biblioteca para acessar e processar a base de dados do TCGA
# https://github.com/ucscXena/xenaPython
!pip install xenaPython
!pip install --upgrade xenaPython
!pip install pandas

In [ ]:
#Conectar Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import xenaPython as xena
import pandas as pd
import os 
import sys
from datetime import datetime
import pytz
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
#Gene escolhido
gene_codigo_escolhido = "ENSG00000135318.11";
gene_escolhido = "NT5E";

#Selecionando a base de dados
#Gene de expressão
hub_gene_expression = "https://toil.xenahubs.net"
dataset_gene_expression = "tcga_RSEM_gene_fpkm"

#Fenotipos
hub_phenotype = "https://pancanatlas.xenahubs.net"
dataset_phenotype = "Survival_SupplementalTable_S1_20171025_xena_sp"

#DNA methylation (Methylation27K)
hub_methylation_27 = "https://tcga.xenahubs.net"
dataset_methylation_27 = "TCGA.PANCAN.sampleMap/HumanMethylation27"
sitios_methylation_27 = ['cg17488985', 'cg17966619']

#DNA methylation (Methylation450K)
hub_methylation_450 = "https://pancanatlas.xenahubs.net"
dataset_methylation_450 = "jhu-usc.edu_PANCAN_HumanMethylation450.betaValue_whitelisted.tsv.synapse_download_5096262.xena"
sitios_methylation_450 = ['cg27039625', 'cg17644557', 'cg13315970', 'cg21730993', 'cg10663055', 'cg17488985', 'cg24635468', 'cg23157089', 'cg17966619', 'cg27297263', 'cg00925339', 'cg23172664', 'cg24702826', 'cg09989847']

gene_encontrado = False
numero_amostras = 0
genes = []
amostras = []

genes = xena.dataset_field (hub_gene_expression, dataset_gene_expression)

#Diretório que será salvo no Google Drive
#verifica em qual sistema o código está rodando
if (sys.platform == 'linux'):
	_bar = '/'
else:
	_bar = '\\'

date_today = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime("%Y.%m.%d-%H:%M:%S")
path_drive = os.getcwd()+_bar+'drive'+_bar+'MyDrive'+_bar+'Colab Notebooks - TCC'+_bar+'Dados_TCGA'+_bar+date_today

# Verifica se existe o gene escolhido
for gene in genes:
  if gene == gene_codigo_escolhido:
    gene_encontrado = True

if gene_encontrado == False:
    print("Gene "+gene_codigo_escolhido+" não foi encontrado!")  
    exit()
else:
    print("Gene "+gene_codigo_escolhido+" foi encontrado!")  

# Verifica o fenotipo correspondente a amostra
fenotipos = xena.dataset_field (hub_phenotype, dataset_phenotype)

# Reune as amostras
numero_amostras = xena.dataset_samples_n_dense_matrix(hub_gene_expression, dataset_gene_expression)
amostras = xena.dataset_samples (hub_gene_expression, dataset_gene_expression, numero_amostras[0])

# Cria as colunas na planilha
colunas = ['Sample', gene_codigo_escolhido]
for fenotipo in fenotipos:
    colunas.append(fenotipo)
for sitio_methylation_450 in sitios_methylation_450:
    colunas.append(sitio_methylation_450)
for sitio_methylation_27 in sitios_methylation_27:
    colunas.append(sitio_methylation_27)
df = pd.DataFrame(columns=colunas)

# Amostras e valor correspondente
pontuacao = xena.dataset_probe_values(hub_gene_expression, dataset_gene_expression, amostras, [gene_codigo_escolhido])
# Fenotipos correspondente a amostra
valores_fenotipos = xena.dataset_probe_values(hub_phenotype, dataset_phenotype, amostras, fenotipos)
# Metilação correspondente (Methylation450K)
valores_methylation_450 = xena.dataset_probe_values(hub_methylation_450, dataset_methylation_450, amostras, sitios_methylation_450)
# Metilação correspondente (Methylation27K)
valores_methylation_27 = xena.dataset_probe_values(hub_methylation_27, dataset_methylation_27, amostras, sitios_methylation_27)

# Verifica as amostras correspondentes ao gene escolhido
for index, amostra in enumerate(amostras):
    # Amostras e valor correspondente
    row = ''
    row = [amostra, pontuacao[1][0][index]]
    
    # Fenotipos correspondente a amostra
    for valor_fenotipo in valores_fenotipos[1]:
       row.append(valor_fenotipo[index])
  
    # Metilação correspondente (Methylation450K)
    for valor_methylation_450 in valores_methylation_450[1]:
        row.append(valor_methylation_450[index])

    # Metilação correspondente (Methylation27K)
    for valor_methylation_27 in valores_methylation_27[1]:
        row.append(valor_methylation_27[index])

    # Incrimenta uma nova linha na planilha
    df.loc[index] = row

# Salva a planilha
df.to_csv(path_drive, sep = ',')        

**Exibe a planilha**

In [ ]:
GPLRawData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.09.12-14:35:16", sep=',')
display(GPLRawData)

GPLRawData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.08.23-00:01:58", sep=',')
display(GPLRawData)

**Realiza a organização dos dados**

Pares e amostras simples

In [ ]:
#Diretório que será salvo no Google Drive
#verifica em qual sistema o código está rodando
if (sys.platform == 'linux'):
	_bar = '/'
else:
	_bar = '\\'

date_today = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime("%Y.%m.%d-%H:%M:%S")
path_drive = os.getcwd()+_bar+'drive'+_bar+'MyDrive'+_bar+'Colab Notebooks - TCC'+_bar+'Dados_TCGA'+_bar+date_today

#Fera duas planilhas, uma com as amostras combinadas e outra com dados simples(Sem combinações)
GPLRawData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.08.23-00:01:58", sep=',')
dfList = GPLRawData
dfPairs = pd.DataFrame(columns=GPLRawData.columns.values.tolist())
dfSimple = pd.DataFrame(columns=GPLRawData.columns.values.tolist())

dfList['leitura'] = 0
dfList['pareamento'] = 0
dfPairs.insert(0, 'pareamento', '')
pareamento = 1

# Função que realiza o agrupamento das amostras
def check_corresponding(sample_exists):
  item_encontrado = []
  for index, item in enumerate(sample_exists):
     if item == True:
       inx = dfList.loc[index]['Sample'][13]+''+dfList.loc[index]['Sample'][14]
       item_encontrado.insert(int(inx), (index, dfList.loc[index]['Sample']))  
  if len(item_encontrado) > 1:
    # Organiza os dados (01, 06, 11..) de forma ordenada
    item_encontrado = sorted(item_encontrado, key = lambda x: (x[1]))    
  return item_encontrado

for index, item in enumerate(dfList.iterrows()):
  #if index < 100:
    sample_exists = False
    sample_exists = dfList ['Sample']. str.contains (item[1]['Sample'][:12], case = False)
    check_corresponding_index = check_corresponding(sample_exists)
    if len(check_corresponding_index) > 1:
      # Quando for encontrado conjunto de combinações
      entrou = 0
      for item_check in check_corresponding_index:
        if dfList.loc[item_check[0]]['leitura'] == 0:
          dfList.loc[item_check[0], 'pareamento'] = str(pareamento)+'.'+item_check[1][13]+item_check[1][14]
          dfPairs.loc[len(dfPairs)] = dfList.loc[item_check[0]]
          dfList.loc[item_check[0], 'leitura'] = 1
          entrou = 1
      if entrou == 1:
        pareamento = pareamento + 1
    else:
      # Quando for encontrado amostras simples(Sem combinações)
      if dfList.loc[check_corresponding_index[0][0]]['leitura'] == 0:
        dfSimple.loc[len(dfSimple)] = dfList.loc[check_corresponding_index[0][0]]
        dfList.loc[check_corresponding_index[0][0], "leitura"] = 1

display(dfSimple) 
display(dfPairs) 

# Salva a planilha
dfSimple.to_csv(path_drive, sep = ',') 
dfPairs.to_csv(path_drive, sep = ',')   

**Estatistica descritiva**

In [ ]:
#Tamanho do arquivo resultante
dir_path = '/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.09.20-01:00:16'
f_path = os.path.join(dir_path)
f_size = os.path.getsize(f_path)
f_size_kb = f_size/1024 # obter resultado em kB
print('Tamanho do arquivo resultante:')
print(str(int(f_size_kb))+' kb')

In [ ]:
#Quantos atributos (Colunas) e quantas estâncias (Linhas) tem na planilha
#GPLRawData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.08.23-00:01:58", sep=',')
GPLRawData = pd.read_csv("/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.09.20-01:00:16", sep=',')
print('Quantidade de linhas e colunas:')
print(str(GPLRawData[GPLRawData.columns[0]].count())+' linhas') 
print(str(len(GPLRawData.columns.values.tolist()))+' colunas') 

In [ ]:
#Quantidade de amostras e tipos tumorais
vetor_planilhas = []
vetor_planilhas.append(["Sample", "/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.08.23-00:01:58", "Planilha original - Quantidade em tipos de amostras:"])
vetor_planilhas.append(["Sample", "/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.09.20-01:00:16", "Planilha pareada - Quantidade em tipos de amostras:"])
vetor_planilhas.append(["cancer type abbreviation", "/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.08.23-00:01:58", "Planilha original - Quantidade de tipos tumorais:"])
vetor_planilhas.append(["cancer type abbreviation", "/content/drive/MyDrive/Colab Notebooks - TCC/Dados_TCGA/2021.09.20-01:00:16", "Planilha pareada - Quantidade de tipos tumorais:"])
coluna_planilhas = ['Tipo', 'Quantidade', 'Percentual']
number_sample_type = [[],[],[],[]]

for idx, item_planilha in enumerate(vetor_planilhas):
  GPLRawData = pd.read_csv(str(item_planilha[1]), sep=',')
  def check_exists(type_sample):
    for index, item in enumerate(number_sample_type[idx]):
      if str(item[0]) == str(type_sample):
        number_sample_type[idx][index][1] = int(number_sample_type[idx][index][1])+1
        return True
    return False
  for index, item in enumerate(GPLRawData.iterrows()):
    if str(item_planilha[0]) == "Sample":
      type_sample = item[1][str(item_planilha[0])][13]+''+item[1][str(item_planilha[0])][14]
    if str(item_planilha[0]) == "cancer type abbreviation":
      type_sample = item[1][item_planilha[0]]

    if check_exists(type_sample) == False:
      number_sample_type[idx].append([type_sample, 1])
  number_sample_type[idx] = sorted(number_sample_type[idx], key = lambda x: (x[0]))
  for index, item in enumerate(number_sample_type[idx]):
    percentual = ((number_sample_type[idx][index][1])/(GPLRawData[GPLRawData.columns[0]].count()))*100
    number_sample_type[idx][index].append(percentual)

  print(item_planilha[2])
  print(str(GPLRawData[GPLRawData.columns[0]].count())+' amostras') 

  df = pd.DataFrame(number_sample_type[idx])
  df.columns = coluna_planilhas
  df = pd.concat([df], ignore_index=True)
  display(df)

  grupos = []
  valores = []
  for index, item in enumerate(number_sample_type[idx]):
    grupos.append(item[0])
    valores.append(item[1])

  plt.bar(grupos, valores)
  plt.show()

  print()
  print()


In [ ]:
#Sítios de metilação
juncao_vetores = [0, 1]
metilacao = ['cg27039625', 'cg17644557', 'cg13315970', 'cg21730993', 'cg10663055', 'cg17488985', 'cg24635468', 'cg23157089', 'cg17966619', 'cg27297263', 'cg00925339', 'cg23172664', 'cg24702826', 'cg09989847', 'cg17488985.1', 'cg17966619.1']
dados_metilacao = []
coluna_planilhas_metilacao = ['Metilação', 'Quantidade', 'Percentual']
metilacao_informacoes = ['Amostras originais', 'Amostras pareadas']

for idx, juncao_vetores_item in enumerate(juncao_vetores):
  df = pd.read_csv(str(vetor_planilhas[juncao_vetores_item][1]), sep=',')
  dados_metilacao.append([])
  for index, metilacao_item in enumerate(metilacao):
    col_one_list = df[str(metilacao_item)].tolist()
    dados_metilacao[idx].append([])
    dados_metilacao[idx][index].append(metilacao_item)
    com_valor = 0
    sem_valor = 0
    for vetor_item in col_one_list:
      if vetor_item > 0:
        com_valor = com_valor + 1
    dados_metilacao[idx][index].append(com_valor)
    dados_metilacao[idx][index].append(((com_valor)/(df[df.columns[0]].count()))*100)

  print(metilacao_informacoes[idx])
  print()
  dfm = pd.DataFrame(dados_metilacao[idx])
  dfm.columns = coluna_planilhas_metilacao
  dfm = pd.concat([dfm], ignore_index=True)
  display(dfm)
  print()
  print()
  number_sample_type.append(dados_metilacao[idx])


    

In [ ]:
#Geração de gráficos
juncao_vetores = [[0, 1], [2, 3], [4, 5]]
juncao_vetores_names = ["Tipos de amostras", "Tipos tumorais", "Níveis de metilação"]

tipo_tumoral_nome = [("1.0", "BLCA"), ("2.0", "BRCA"), ("3.0", "CESC"), ("4.0", "CHOL"), ("5.0", "COAD"), ("7.0", "ESCA"), ("8.0", "GBM"), ("9.0", "HNSC"), ("10.0", "KICH"), ("11.0", "KIRC"), ("12.0", "KIRP"), ("14.0", "LGG"), ("15.0", "LIHC"), ("16.0", "LUAD"), ("17.0", "LUSC"), ("19.0", "OV"), ("20.0", "PAAD"), ("21.0", "PCPG"), ("22.0", "PRAD"), ("23.0", "READ"), ("24.0", "SARC"), ("25.0", "SKCM"), ("26.0", "STAD"), ("27.0", "TGCT"), ("28.0", "THCA"), ("29.0", "THYM"), ("30.0", "UCEC")]
n_tumoral_nome = []

def check_exists(type_sampl, index):
  for index, item in enumerate(number_sample_type[index]):
    if str(item[0]) == str(type_sampl):
      return item[1]
  return 0
for idx, juncao_vetores_item in enumerate(juncao_vetores):
  juncao_vetores_tipo = []
  valores_produto_A = []
  valores_produto_B = []

  for index, item in enumerate(number_sample_type[juncao_vetores_item[0]]):
    juncao_vetores_tipo.append(item[0])
    valores_produto_A.append(item[1])
    valores_produto_B.append(check_exists(item[0], juncao_vetores_item[1]))

  # Cria eixo x para produto A e produto B com uma separação de 0.25 entre as barras
  fig=plt.figure(figsize=(25, 10), dpi= 80, facecolor='w', edgecolor='k')
  x1 =  np.arange(len(valores_produto_A))
  
  x2 = [x + 0.40 for x in x1]

  # Plota as barras
  plt.bar(x1, valores_produto_A, width=0.40, label = 'Amostras originais', color = 'blue')
  plt.bar(x2, valores_produto_B, width=0.40, label = 'Amostras pareadas', color = 'red')

  # coloca o nome dos meses como label do eixo x
  nome_tumor = ""
  for name_index, name_item in enumerate(tipo_tumoral_nome):
    if name_item[0] == str(name_type[x_index]):
      nome_tumor = name_type[x_index]+" - "+name_item[1]
  plt.xticks([x + 0.45 for x in range(len(valores_produto_A))], juncao_vetores_tipo)

  # inseri uma legenda no gráfico
  plt.legend()

  plt.title(juncao_vetores_names[idx])

  plt.show()

In [ ]:
dfList = GPLRawData

dfList['leitura'] = 0
lista_pareada = []

# Função que realiza o agrupamento das amostras
def check_corresponding(sample_exists):
  item_encontrado = []
  for index, item in enumerate(sample_exists):
     if item == True:
       inx = dfList.loc[index]['Sample'][13]+''+dfList.loc[index]['Sample'][14]
       item_encontrado.insert(int(inx), (index, dfList.loc[index]['Sample']))  
  if len(item_encontrado) > 1:
    # Organiza os dados (01, 06, 11..) de forma ordenada
    item_encontrado = sorted(item_encontrado, key = lambda x: (x[1]))    
  return item_encontrado
def remove_repetidos(lista):
    l = []
    for i in lista:
        if i not in l:
            l.append(i)
    l.sort()
    return l
for index, item in enumerate(dfList.iterrows()):
  sample_exists = False
  sample_exists = dfList ['Sample']. str.contains (item[1]['Sample'][:12], case = False)
  check_corresponding_index = check_corresponding(sample_exists)
  if len(check_corresponding_index) > 1:
    # Quando for encontrado conjunto de combinações
    lista_pareada.append(check_corresponding_index)
lista_pareada = remove_repetidos(lista_pareada)
print(lista_pareada)

In [ ]:
#Eliminando amostras que não serão utilizadas

lista_pareada_novo = []

for x_index, x_item in enumerate(lista_pareada):
  tamanho_x_item = len(x_item)
  amostra_final = []
  for y_index, y_item in enumerate(x_item):
    amostra_final.append(y_item[1][13:])  
  if tamanho_x_item == 3:
    if '01' in amostra_final and '06' in amostra_final and '11' in amostra_final:
      lista_pareada[x_index].pop(amostra_final.index('06'))
      lista_pareada_novo.append(lista_pareada[x_index])
      continue
  elif tamanho_x_item == 2:
    if ('05' in amostra_final and '11' in amostra_final):
      lista_pareada_novo.append(lista_pareada[x_index])
      continue
    elif ('01' in amostra_final and '11' in amostra_final):
      lista_pareada_novo.append(lista_pareada[x_index])
      continue

lista_pareada = lista_pareada_novo
print(lista_pareada)

df_temp = pd.DataFrame(columns=GPLRawData.columns.values.tolist())
df_temp_index = 0
for x_index, x_item in enumerate(lista_pareada):
  for y_index, y_item in enumerate(x_item):
    df_temp.loc[df_temp_index] = GPLRawData.loc[y_item[0]]
    df_temp_index = df_temp_index + 1

#Diretório que será salvo no Google Drive
#verifica em qual sistema o código está rodando
if (sys.platform == 'linux'):
	_bar = '/'
else:
	_bar = '\\'

date_today = datetime.now(pytz.timezone('America/Sao_Paulo')).strftime("%Y.%m.%d-%H:%M:%S")
path_drive = os.getcwd()+_bar+'drive'+_bar+'MyDrive'+_bar+'Colab Notebooks - TCC'+_bar+'Dados_TCGA'+_bar+date_today
  
GPLRawData = df_temp 

# Salva a planilha
GPLRawData.to_csv(path_drive, sep = ',')

In [ ]:
#Organiza os dados para gerar o Boxplot de cada tipo tumoral
import plotly.graph_objects as go
df_cancer_type_abbreviation = GPLRawData
type_abbreviation = number_sample_type[3]

name_type = []
values_type = []
for x_index, x_item in enumerate(type_abbreviation):
  if str(x_item[0]) != str('nan'):
    df_temp = pd.DataFrame(columns=GPLRawData.columns.values.tolist())
    name_type.append(str(x_item[0]))
    values_type.append([[],[]])
    for y_index, y_item in df_cancer_type_abbreviation.iterrows():
      if y_item['cancer type abbreviation'] == x_item[0]:
        df_temp.loc[y_index] = y_item
        type_indice = y_item['Sample'][13]+y_item['Sample'][14]
        if type_indice == '01' or type_indice == '05':
          values_type[x_index][1].append(y_item['ENSG00000135318.11'])
        if type_indice == '11':
          values_type[x_index][0].append(y_item['ENSG00000135318.11'])
    #display(df_temp)
print(name_type)
print(values_type)

In [ ]:
#Gerando o Boxplot de cada tipo tumoral

tipo_tumoral_nome = [("1.0", "BLCA"), ("2.0", "BRCA"), ("3.0", "CESC"), ("4.0", "CHOL"), ("5.0", "COAD"), ("7.0", "ESCA"), ("8.0", "GBM"), ("9.0", "HNSC"), ("10.0", "KICH"), ("11.0", "KIRC"), ("12.0", "KIRP"), ("14.0", "LGG"), ("15.0", "LIHC"), ("16.0", "LUAD"), ("17.0", "LUSC"), ("19.0", "OV"), ("20.0", "PAAD"), ("21.0", "PCPG"), ("22.0", "PRAD"), ("23.0", "READ"), ("24.0", "SARC"), ("25.0", "SKCM"), ("26.0", "STAD"), ("27.0", "TGCT"), ("28.0", "THCA"), ("29.0", "THYM"), ("30.0", "UCEC")]
n_tumoral_nome = []

colors = ['royalblue', 'indianred']
name_description = ['Não tumoral', 'Tumoral']

for x_index, x_item in enumerate(values_type):
  n_tumoral_nome.append(len(x_item[0]))
  fig = go.Figure()

  for xd, yd, cls in zip(name_description, values_type[x_index], colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            marker_size=7,
            line_width=1)
        )

  nome_tumor = ""
  for name_index, name_item in enumerate(tipo_tumoral_nome):
    if name_item[0] == str(name_type[x_index]):
      nome_tumor = name_item[1]
    
  fig.update_layout(
    title=('Tipo tumoral: '+name_type[x_index]+" - "+nome_tumor),
    xaxis=dict(title=('N = '+str(len(x_item[0]))), zeroline=False),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=True
  )

  fig.show()

  print()

In [ ]:
#Estatística descritiva e análise estatística das amostras pareadas
df_temp = pd.DataFrame(columns=['Amostra', 'P VALUE', 'N', 'NT_mean', 'NT_median', 'NT_sd', 'NT_min', 'NT_max', 'T_mean', 'T_median', 'T_sd', 'T_min', 'T_max'])

import statistics
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t

def independent_ttest(dataNTumoral, dataTumoral):
  mean1, mean2 = mean(dataNTumoral), mean(dataTumoral)
  se1, se2 = sem(dataNTumoral), sem(dataTumoral)
  sed = sqrt(se1**2.0 + se2**2.0)
  t_stat = (mean1 - mean2) / sed
  df = len(dataNTumoral) + len(dataTumoral) - 2
  p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
  return p

def p_value(dataNTumoral, dataTumoral):
  if dataNTumoral != [] and dataTumoral != []:
    p = independent_ttest(dataNTumoral, dataTumoral)
    return ('%.4f' % (p))
  else:
    return ''

for x_index, x_item in enumerate(name_type):
  nome_tumor = ''
  for name_index, name_item in enumerate(tipo_tumoral_nome):
    if name_item[0] == str(name_type[x_index]):
      nome_tumor = name_item[1]

  dataNTumoral = values_type[x_index][0]
  dataTumoral = values_type[x_index][1]
  # p-value
  P_value =  p_value(dataNTumoral, dataTumoral)
  # Não tumoral
  NT_mean = statistics.mean(dataNTumoral) if dataNTumoral != [] else ''
  NT_median = statistics.median(dataNTumoral) if dataNTumoral != [] else ''
  NT_sd = np.std(dataNTumoral) if dataNTumoral != [] else ''
  NT_min = min(dataNTumoral) if dataNTumoral != [] else ''
  NT_max = max(dataNTumoral) if dataNTumoral != [] else ''
  # Tumoral
  T_mean = statistics.mean(dataTumoral) if dataTumoral != [] else ''
  T_median = statistics.median(dataTumoral) if dataTumoral != [] else ''
  T_sd = np.std(dataTumoral) if dataTumoral != [] else ''
  T_min = min(dataTumoral) if dataTumoral != [] else ''
  T_max = max(dataTumoral) if dataTumoral != [] else ''

  df_temp.loc[x_index] = [(x_item+' - '+nome_tumor), P_value, n_tumoral_nome[x_index], NT_mean, NT_median, NT_sd, NT_min, NT_max, T_mean, T_median, T_sd, T_min, T_max]  # adding a row

display(df_temp)

In [ ]:
# Seleção dos tipos tumorais que possuem relavância para a aplicação de machine learning
# Removendo os tipos tumorais sem relevância, ou seja, tipos tumorais sem pares.
for x_index, x_item in enumerate(df_temp.iterrows()):
  if int(x_item[1]['N']) == 0:
    df_temp = df_temp.drop(x_index, axis=0)
  elif float(x_item[1]['P VALUE']) > 0.01:
    df_temp = df_temp.drop(x_index, axis=0)
# Ordenando os tipos tumorais através do número de pares. 
df_temp_p_value = df_temp.sort_values(by='N', ascending=False)
df_tipos_tumorais_relavantes = df_temp_p_value
display(df_tipos_tumorais_relavantes)


In [ ]:
#Gerando o Boxplot de cada tipo tumoral
import seaborn as sns
sns.set_theme(style="whitegrid")
tips = sns.load_dataset("tips")

tipo_tumoral_nome = [("1.0", "BLCA"), ("2.0", "BRCA"), ("3.0", "CESC"), ("4.0", "CHOL"), ("5.0", "COAD"), ("7.0", "ESCA"), ("8.0", "GBM"), ("9.0", "HNSC"), ("10.0", "KICH"), ("11.0", "KIRC"), ("12.0", "KIRP"), ("14.0", "LGG"), ("15.0", "LIHC"), ("16.0", "LUAD"), ("17.0", "LUSC"), ("19.0", "OV"), ("20.0", "PAAD"), ("21.0", "PCPG"), ("22.0", "PRAD"), ("23.0", "READ"), ("24.0", "SARC"), ("25.0", "SKCM"), ("26.0", "STAD"), ("27.0", "TGCT"), ("28.0", "THCA"), ("29.0", "THYM"), ("30.0", "UCEC")]
n_tumoral_nome = []

colors = ['royalblue', 'indianred']
name_description = ['Não tumoral', 'Tumoral']

for x_index, x_item in enumerate(values_type):
  nome_tumor = ""
  for name_index, name_item in enumerate(tipo_tumoral_nome):
    if name_item[0] == str(name_type[x_index]):
      nome_tumor = name_type[x_index]+" - "+name_item[1]

  
  data = {'Não tumoral':  values_type[x_index][0],
        'Tumoral': values_type[x_index][1]}
  df = pd.DataFrame(data)
  
  result = df_tipos_tumorais_relavantes['Amostra'].str.contains("2.0 - BRCA")
  exists = nome_tumor in df_tipos_tumorais_relavantes.values
  if exists == True:
    plt.figure( figsize=(5, 6))
    sns.boxplot(x="variable", y="value", data=pd.melt(df), palette={"Não tumoral": "#2b76ca", "Tumoral": "#b10026"})
    sns.swarmplot(x="variable", y="value", data=pd.melt(df), color="#140f07")
    plt.title(nome_tumor, loc="left")
    plt.show()

In [ ]:
# Removendo colunas não utilizados e adicionando o diagnóstico a cada tipo tumoral
df_temp_colunas_selecionadas = ['Sample', 'ENSG00000135318.11', 'cancer type abbreviation']
for x_item in metilacao:
  df_temp_colunas_selecionadas.append(x_item)

df_temp = GPLRawData[df_temp_colunas_selecionadas]
df_temp['Tipo de amostra'] = ''

for x_index, x_item in df_temp.iterrows():
  type_indice = x_item['Sample'][13]+x_item['Sample'][14]
  if type_indice == '01' or type_indice == '05':
    df_temp.loc[x_index, 'Tipo de amostra'] = 'Amostra Tumoral'
  elif type_indice == '11':
    df_temp.loc[x_index, 'Tipo de amostra'] = 'Amostra Normal'
display(df_temp)
GPLRawData = df_temp

**Machine Learning**

In [ ]:
# Separação das amostras por tipo tumoral conforme a seleção e os testes realizados
# Aplicação do machine learning em cada tipo tumoral

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.metrics import RocCurveDisplay

from pandas import DataFrame

import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

methylation_genes = ['ENSG00000135318.11', 'cg27039625', 'cg17644557', 'cg13315970', 'cg21730993', 'cg10663055', 'cg17488985', 'cg24635468', 'cg23157089', 'cg17966619', 'cg27297263', 'cg00925339', 'cg23172664', 'cg24702826', 'cg09989847','cg17488985.1', 'cg17966619.1']

# Tumor específico ou 'all' para analisar todos os tipos tumorais
tumor_type = ['THCA']
# Escolha da técnica utilizada (holdout_simple ou holdout_validation)
model_validation_techniques = "holdout_simple"
random_state_parameter = 14
test_size_paramete = 0.5

# Criando um data frame para cada tipo de métrica
# Métricas (accuracy, precision, recall e f1)
df_temp = methylation_genes.copy()
df_temp.insert(0, 'tipo tumoral')
df_accuracy = pd.DataFrame(columns=df_temp)
df_precision = pd.DataFrame(columns=df_temp)
df_recall = pd.DataFrame(columns=df_temp)
df_f1 = pd.DataFrame(columns=df_temp)


def tipo_tumoral_random_forest(df_tipo_tumoral, nomenclatura_tipo_tumoral, tumor_index):
  # Valida o tipo tumoral escolhido para a análise
  nomenclatureSplit = nomenclatura_tipo_tumoral.split('-')
  if tumor_type != 'all' and nomenclatureSplit[1].strip(" ") not in tumor_type:
    return

  print('Tipo tumoral: '+nomenclatura_tipo_tumoral)
  for index, gene in enumerate(methylation_genes):
    # Cria a tabelas temporárias para cada análise (Gene e (Gene + metilação))
    df_temp = create_df_temp(df_tipo_tumoral, gene)
    # Remove os items apenas dos data fremes temporário que tiverem metilação inclusa
    if index > 0:
      df_temp = remove_samples_no_value(df_temp)
    # Caso não tiver amostras, utiliza um 0 nos data frames de métricas
    if len(df_temp) == 0:
      insert_value_metrics(tumor_index, gene, [0,0,0,0])
      continue
    # Machine learning para cada combinação e tipo tumoral
    applying_machine_learning(df_temp, index, gene, nomenclatura_tipo_tumoral, tumor_index)

# Criando as seleções de genes
def create_df_temp(df_tipo_tumoral, current_gene):
  selected_columns = ['ENSG00000135318.11']
  if current_gene != selected_columns[0]:
    selected_columns.insert(1, current_gene)
  selected_columns.insert(2, 'Tipo de amostra')
  selected_columns.insert(3, 'Sample')
  df_temp = df_tipo_tumoral[selected_columns]
  return df_temp

# Removendo amostras que não possuem um valor (nan)
def remove_samples_no_value(df_gene_selection):
  for index, row in df_gene_selection.iterrows():
    if str(row[1]) == str('nan'):
      df_gene_selection = df_gene_selection.drop(index)
  return df_gene_selection

# Aplicação do machine learning
def applying_machine_learning(df_gene_selection, index, gene, nomenclatura_tipo_tumoral, tumor_index):
  X_columns = ['ENSG00000135318.11'] if index == 0 else ['ENSG00000135318.11', gene]
  X = df_gene_selection[X_columns]
  y = df_gene_selection['Tipo de amostra']

  # Método Hold-out de forma simples, sem uma nova divisão do treinamento em validação
  if model_validation_techniques == 'holdout_simple':
    X_train, X_test, y_train, y_test, y_pred = holdout_simple(X, y)
  
  # Método Hold-out com uma nova divisão do treinamento em validação
  if model_validation_techniques == 'holdout_validation':
    X_train, X_test, y_train, y_test, y_pred = holdout_with_validation(X, y)

  # metricas resultantes
  value_metrics = [
      metrics.accuracy_score(y_test, y_pred),
      metrics.precision_score(y_test, y_pred, average='macro'),
      metrics.recall_score(y_test, y_pred, average='macro'),
      metrics.f1_score(y_test, y_pred, average='macro')
  ]

  # Incluindo o tipo tumoral nos data frames
  insert_tumor_type(index, tumor_index, nomenclatura_tipo_tumoral)
  # Incluindo os valores(métricas) nos data frames
  insert_value_metrics(tumor_index, gene, value_metrics)

  # Curva Roc
  # Amostras normais
  roc_curve(df_gene_selection, tumor_index, index, gene, X_train, y_train, X_test, y_test, "Amostra Normal", "Normal samples")
  # Amostras tumorais
  roc_curve(df_gene_selection, tumor_index, index, gene, X_train, y_train, X_test, y_test, "Amostra Tumoral", "Tumor samples")

# Hold-out simples
def holdout_simple(X, y):
  # 50% training and 50% test
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_paramete, random_state = random_state_parameter)
  clf=RandomForestClassifier(n_estimators=100)
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  return X_train, X_test, y_train, y_test, y_pred

# Hold-out com a divisão do treinamento em um novo grupo de validação
def holdout_with_validation(X, y):
  print('...')

# Gerando a curva roc para cada análise
def roc_curve(df_gene_selection, tumor_index, index, gene, X_train, y_train, X_test, y_test, pos_label, ylabel):
  rows_count = len(X_train.index)
  # Gera a curva apenas para o tipo tumral THCA
  if rows_count <= 15 or tumor_index != 2:
    return 
  description = gene if index == 0 else "ENSG00000135318.11 + "+gene
  print()

  clf = SVC(random_state=random_state_parameter).fit(X_train, y_train)

  RocCurveDisplay.from_estimator(clf, X_test, y_test, pos_label=pos_label)
  plt.title(description)
  plt.xlabel('')
  plt.ylabel(ylabel)
  plt.show()

# Inserindo os tipos tumorais dentro das métricas
def insert_tumor_type(index, tumor_index, nomenclatura_tipo_tumoral):
  if index == 0:
    df_accuracy.loc[tumor_index, 'tipo tumoral'] = nomenclatura_tipo_tumoral
    df_precision.loc[tumor_index, 'tipo tumoral'] = nomenclatura_tipo_tumoral
    df_recall.loc[tumor_index, 'tipo tumoral'] = nomenclatura_tipo_tumoral
    df_f1.loc[tumor_index, 'tipo tumoral'] = nomenclatura_tipo_tumoral

# Inserido os resultados em cada métrica
def insert_value_metrics(tumor_index, gene, value_metrics):
  df_accuracy.loc[tumor_index, str(gene)] = value_metrics[0]
  df_precision.loc[tumor_index, str(gene)] = value_metrics[1]
  df_recall.loc[tumor_index, str(gene)] = value_metrics[2]
  df_f1.loc[tumor_index, str(gene)] = value_metrics[3]

# Seleção por tipo tumoral
index = 0
for x_index, x_item in enumerate(df_tipos_tumorais_relavantes.iterrows()):
  df_tipo_tumoral = False
  df_tipo_tumoral = pd.DataFrame(columns=GPLRawData.columns.values.tolist())
  tipo_tumoral_relevante = x_item[1]['Amostra'].split('-')

  # Seleção das amostras através do tipo tumoral (x_item)
  tipo_tumoral_relevante_index = 0
  for y_index, y_item in enumerate(GPLRawData.iterrows()):
    if str(tipo_tumoral_relevante[0].strip(" ")) == str(y_item[1]['cancer type abbreviation']):
      df_tipo_tumoral.loc[tipo_tumoral_relevante_index] = GPLRawData.loc[y_index]
      tipo_tumoral_relevante_index = tipo_tumoral_relevante_index + 1
  tipo_tumoral_random_forest(df_tipo_tumoral, x_item[1]['Amostra'], index)
  index=index+1

**Heat map**

In [ ]:
def delete_column(df_metrics, name_column):
  if name_column in df_metrics.columns:
    df_metrics = df_metrics.drop(name_column, 1)
  return df_metrics

def delete_row(df_metrics, name_row):
  metrics_row = df_metrics[df_metrics["tipo tumoral"]==name_row].index
  df_metrics = df_metrics.drop(metrics_row)
  return df_metrics  

def display_heat_map(df_metrics, type_metrics):
  print('')
  print(type_metrics)
  
  df_metrics = delete_column(df_metrics, 'cg17488985.1')
  df_metrics = delete_column(df_metrics, 'cg17966619.1')
  df_metrics = delete_row(df_metrics, '10.0 - KICH') 

  df_tumor_types = df_metrics['tipo tumoral'].values.tolist()
  df_metrics = delete_column(df_metrics, 'tipo tumoral')
  
  df_cols = ['ENSG00000135318.11', 'cg27039625', 'cg17644557', 'cg13315970', 'cg21730993', 'cg10663055', 'cg17488985', 'cg24635468', 'cg23157089', 'cg17966619', 'cg27297263', 'cg00925339', 'cg23172664', 'cg24702826', 'cg09989847']
  
  df_formatted = pd.DataFrame(df_metrics.values.tolist(), index=df_tumor_types, columns=df_cols)
  plt.figure(1, figsize=(20, 13))
  cmap = sns.cm.rocket_r
  sns.heatmap(df_formatted, annot=True, cmap=cmap)
  plt.show()

display_heat_map(df_accuracy, 'Accuracy')
display_heat_map(df_precision, 'Precision')
display_heat_map(df_recall, 'Recall')
display_heat_map(df_f1, 'F1')